In [61]:
import numpy as np
import os
import nibabel as nib

from scipy import ndimage
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
import time
import copy
import torch
from torch.utils.data import DataLoader

In [62]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True, source='github', force_reload=True)
device = torch.device('cpu')
# torch.cuda.is_available()


Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to C:\Users\pjame/.cache\torch\hub\main.zip


In [63]:
model

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 

In [64]:
for param in model.parameters():
    param.requires_grad = False

new_layer = torch.nn.Sequential(
    torch.nn.Linear(2048, 4, dtype=torch.float32),
    torch.nn.AdaptiveAvgPool3d(output_size=1)
)

model._modules['blocks'][-1] = new_layer 
model._modules['blocks'][-1].requires_grad = True

In [65]:
model

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 

In [66]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, filename):
      self.data = torch.load(filename)
      self.X, self.Y = self.data
      
  def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

  def __getitem__(self, index):
      x = self.X[index]
      y = self.Y[index]
      return x,y


In [68]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                print(inputs.dtype)
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        print(inputs.dtype)
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        print(inputs.dtype)
                        outputs = model(inputs)
                        
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [69]:

training_dataset = Dataset('training_dataset.pt')
testing_dataset = Dataset('testing_dataset.pt')


In [70]:
train_loader = DataLoader(training_dataset)

dataloaders = {
    'train': DataLoader(training_dataset, batch_size=10, shuffle=True),
    'test': DataLoader(testing_dataset, batch_size=10, shuffle=False)
}

In [71]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, dataloaders, torch.nn.CrossEntropyLoss, optimizer)

Epoch 0/24
----------
torch.float32
torch.float32


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2097152x2 and 2048x4)

In [ ]:
def create_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""
    print(width, height, depth)
    inputs = keras.Input((width, height, depth, 3))
    
    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="softmax")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    print('done')
    return model

def deserialize(serialized_example):
    feature_map = {
        'x_shape': tf.io.FixedLenFeature([3], dtype=tf.int64),
        'x': tf.io.FixedLenFeature([], dtype=tf.string),
        'y': tf.io.FixedLenFeature([], dtype=tf.string)
    }

    example = tf.io.parse_single_example(serialized_example, feature_map)

    x_shape = example['x_shape']
    x_bytes = example['x']
    y_bytes = example['y']

    x = tf.cast(x, tf.float32)
    y = tf.io.parse_tensor(y_bytes, out_type=tf.float32)
    y = tf.cast(y, tf.int64)
    return x, y

In [ ]:
train_data = tf.data.TFRecordDataset('training_dataset.tfrecord')



: 

: 

In [ ]:
for record in train_data:
    x, y = deserialize(record)
    print(x.shape)
    print(y)
    print(x)
    break


KeyboardInterrupt



In [ ]:
train_data = tf.data.TFRecordDataset('training_dataset.tfrecord')
train_data = train_data.map(deserialize)
train_data = train_data.shuffle(10)
train_data = train_data.batch(5)
train_data = train_data.prefetch(5)



In [ ]:


model = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=4,
)


initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics='accuracy')

batch_size = 1
epochs = 2
validation_split = 0.1

with tf.device("CPU"):
    model.fit(
        train_data,
        epochs=epochs,
        verbose=2
    )



128 128 64
done
Epoch 1/2


ValueError: in user code:

    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:862 train_function  *
        return step_function(self, iterator)
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:852 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\Users\pjame\.conda\envs\tf-gpu\lib\site-packages\tensorf